In [ ]:
# %% [markdown]
# # Αναγνώριση Είδους και Εξαγωγή Συναισθήματος από Μουσική

# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# %% [code]
import librosa.display
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import os 

import copy
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from torch.utils.data import SubsetRandomSampler, DataLoader
import re

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset

# %% [code]
# Keep all the train_labels information
train_labels = pd.read_csv('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/train_labels.txt')

# List with just the label 
train_label=[]
for i in range(train_labels.shape[0]):
    train_label.append(str(train_labels.iloc[i]).split('\\')[1].split(' ')[4])

def find_music_gender(num):
    track1_pos = train_label.index(num +'.fused.full.npy.gz')
    label1 = str(train_labels.iloc[track1_pos]).split('\\')[2].split(' ')[0][1:-6]
    return label1
    #print('label1:',label1, '\nlabel2:',label2)

# %% [markdown]
# > # Step 1 [a-c]

# %% [code]
spec1 = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/train/10079.fused.full.npy')
print('spec shape:',spec1.shape, 'Music Gender:', find_music_gender('10079'))

spec2 = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/train/10033.fused.full.npy')
print('spec shape:',spec2.shape, 'Music Gender:', find_music_gender('10033'))


mel1, spec1 = spec1[:128], spec1[128:]
mel2, spec2 = spec2[:128], spec2[128:]


# PLOTS 
plt.figure(figsize = (10,6))

plt.subplot(2, 1, 1)
librosa.display.specshow(mel1, y_axis='linear')
plt.colorbar(format='%+1.0f dB')
plt.title(find_music_gender('10079'))

plt.subplot(2, 1, 2)
librosa.display.specshow(mel2, y_axis='linear')
plt.colorbar(format='%+1.0f dB')
plt.title(find_music_gender('10033'))

# %% [markdown]
# # Step 2 [a-b]

# %% [code]
spec1_beat = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/train/10079.fused.full.npy')
print('spec shape:',spec1_beat.shape, 'Music Gender:', find_music_gender('10079'))

spec2_beat = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/train/10033.fused.full.npy')
print('spec shape:',spec2_beat.shape, 'Music Gender:', find_music_gender('10033'))

mel1_beat, spec1_beat = spec1_beat[:128], spec1_beat[128:]
mel2_beat, spec2_beat = spec2_beat[:128], spec2_beat[128:]


plt.figure(figsize = (10,6))

plt.subplot(2, 1, 1)
librosa.display.specshow(mel1_beat, y_axis='linear')
plt.colorbar(format='%+1.0f dB')
plt.title('Beat ' +find_music_gender('10079'))

plt.subplot(2, 1, 2)
librosa.display.specshow(mel2_beat, y_axis='linear')
plt.colorbar(format='%+1.0f dB')
plt.title('Beat ' +find_music_gender('10033'))

# %% [markdown]
# 

# %% [markdown]
# # Step 3

# %% [code]
plt.figure(figsize = (20,10))

plt.subplot(2, 2, 1)
librosa.display.specshow(spec1, y_axis='chroma')
plt.colorbar(format='%+1.0f dB')
plt.title('Chromagrams '+find_music_gender('10079'))

plt.subplot(2, 2, 2)
librosa.display.specshow(spec2, y_axis='chroma')
plt.colorbar(format='%+1.0f dB')
plt.title('Chromagrams '+find_music_gender('10079'))

plt.subplot(2, 2, 3)
librosa.display.specshow(spec1_beat, y_axis='chroma')
plt.colorbar(format='%+1.0f dB')
plt.title('Chromagrams Beat ' +find_music_gender('10033'))

plt.subplot(2, 2, 4)
librosa.display.specshow(spec2_beat, y_axis='chroma')
plt.colorbar(format='%+1.0f dB')
plt.title('Chromagrams Beat ' +find_music_gender('10033'))

# %% [markdown]
# # Step 4 [a-c]

# %% [code]
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from torch.utils.data import SubsetRandomSampler, DataLoader
import re

#split dataset --create train and validation set 
def torch_train_val_split(
        dataset, batch_train, batch_eval,
        val_size=.2, shuffle=True, seed=None):
    #20% for validation
    
    dataset_size = len(dataset)
    indices = list(range(dataset_size)) #indices for train and val
    val_split = int(np.floor(val_size * dataset_size))
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(indices)
    train_indices = indices[val_split:]
    val_indices = indices[:val_split]

    # Sampling elements using the above indices
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
    #we use the custom sampler
    train_loader = DataLoader(dataset,
                              batch_size=batch_train,
                              drop_last=True,
                              sampler=train_sampler)
    val_loader = DataLoader(dataset,
                            batch_size=batch_eval,
                            drop_last=True,
                            sampler=val_sampler)
    return train_loader, val_loader

def read_fused_spectrogram(spectrogram_file):
    spectrogram = np.load(spectrogram_file)
    return spectrogram.T

def read_mel_spectrogram(spectrogram_file):
    spectrogram = np.load(spectrogram_file)[:128]
    return spectrogram.T

    
def read_chromagram(spectrogram_file):
    spectrogram = np.load(spectrogram_file)[128:]
    return spectrogram.T


#Padding: bring all samples to the same length
class PaddingTransform(object):
    def __init__(self, max_length, padding_value=0):
        self.max_length = max_length
        self.padding_value = padding_value

    def __call__(self, s):
        if len(s) == self.max_length:
            return s

        if len(s) > self.max_length:
            return s[:self.max_length]

        if len(s) < self.max_length:
            s1 = copy.deepcopy(s)
            pad = np.zeros((self.max_length - s.shape[0], s.shape[1]), dtype=np.float32)
            s1 = np.vstack((s1, pad))
            return s1

class SpectrogramDataset(Dataset):
    def __init__(self, path, task=None, train=True, max_length=-1, read_spec_fn=read_mel_spectrogram):
        t = 'train' if train else 'test'
        p = os.path.join(path, t)
        self.index = os.path.join(path, "{}_labels.txt".format(t))
        self.files, labels = self.get_files_labels(self.index, task)
        self.feats = [read_spec_fn(os.path.join(p, f)) for f in self.files]
        self.feat_dim = self.feats[0].shape[1]
        self.lengths = [len(i) for i in self.feats]
        self.max_length = max(self.lengths) if max_length <= 0 else max_length
        self.zero_pad_and_stack = PaddingTransform(self.max_length)
        if isinstance(labels, (list, tuple)):
            self.labels = np.array(np.array(labels).astype('float'))

    def get_files_labels(self, txt, task):
        with open(txt, 'r') as fd:
            lines = [l.rstrip().split(',') for l in fd.readlines()[1:]]
        files, labels = [], []
        for l in lines:
            if task=='valence':
                label = l[1]
            elif task=='energy':
                label = l[2]
            else:
                label = l[3]
            # Kaggle automatically unzips the npy.gz format so this hack is needed
            _id = l[0]
            npy_file = '{}.fused.full.npy'.format(_id)
            files.append(npy_file)
            labels.append(label)
        return files, labels

    def __getitem__(self, item):
        # TODO: Inspect output and comment on how the output is formatted
        l = min(self.lengths[item], self.max_length)
        return self.zero_pad_and_stack(self.feats[item]), self.labels[item], l

    def __len__(self): #returns lenght
        return len(self.labels)

# %% [code]
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from torch.utils.data import SubsetRandomSampler, DataLoader
import re

# New classes to be combined 
class_mapping = {
    'Rock': 'Rock',
    'Psych-Rock': 'Rock',
    'Indie-Rock': None,
    'Post-Rock': 'Rock',
    'Psych-Folk': 'Folk',
    'Folk': 'Folk',
    'Metal': 'Metal',
    'Punk': 'Metal',
    'Post-Punk': None,
    'Trip-Hop': 'Trip-Hop',
    'Pop': 'Pop',
    'Electronic': 'Electronic',
    'Hip-Hop': 'Hip-Hop',
    'Classical': 'Classical',
    'Blues': 'Blues',
    'Chiptune': 'Electronic',
    'Jazz': 'Jazz',
    'Soundtrack': None,
    'International': None,
    'Old-Time': None
}



class LabelTransformer(LabelEncoder):
    def inverse(self, y):
        try:
            return super(LabelTransformer, self).inverse_transform(y)
        except:
            return super(LabelTransformer, self).inverse_transform([y])

    def transform(self, y):
        try:
            return super(LabelTransformer, self).transform(y)
        except:
            return super(LabelTransformer, self).transform([y])


class SpectrogramDatasetCategory(Dataset):
    def __init__(self, path, class_mapping=None, train=True, max_length=-1, read_spec_fn=read_mel_spectrogram):
        t = 'train' if train else 'test'
        p = os.path.join(path, t)
        self.index = os.path.join(path, "{}_labels.txt".format(t))
        self.files, labels = self.get_files_labels(self.index, class_mapping)
        self.feats = [read_spec_fn(os.path.join(p, f)) for f in self.files]
        self.feat_dim = self.feats[0].shape[1]
        self.lengths = [len(i) for i in self.feats]
        self.max_length = max(self.lengths) if max_length <= 0 else max_length
        self.zero_pad_and_stack = PaddingTransform(self.max_length)
        self.label_transformer = LabelTransformer()
        if isinstance(labels, (list, tuple)):
            self.labels = np.array(self.label_transformer.fit_transform(labels)).astype('int64')

    def get_files_labels(self, txt, class_mapping):
        with open(txt, 'r') as fd:
            lines = [l.rstrip().split('\t') for l in fd.readlines()[1:]]
        files, labels = [], []
        for l in lines:
            label = l[1]
            if class_mapping:
                label = class_mapping[l[1]]
            if not label:
                continue
            # Kaggle automatically unzips the npy.gz format so this hack is needed
            _id = l[0].split('.')[0]
            npy_file = '{}.fused.full.npy'.format(_id)
            files.append(npy_file)
            labels.append(label)
        return files, labels

    def __getitem__(self, item):
        # TODO: Inspect output and comment on how the output is formatted
        l = min(self.lengths[item], self.max_length)
        return self.zero_pad_and_stack(self.feats[item]), self.labels[item], l

    def __len__(self):
        return len(self.labels)

# %% [code]
train_labels=(pd.read_csv('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/train_labels.txt',dtype='str'))

train_labels_classes=[]
for i in range(train_labels.shape[0]):
    train_labels_classes.append(str(train_labels.iloc[i]).split('\\')[2].split(' ')[0][1:-6])

unique_classes_before =set(train_labels_classes)
unique_classes_after =set(class_mapping.values())
unique_classes_after.remove(None)

samplesPerClass_before = {s:0 for s in unique_classes_before}
samplesPerClass_after = {s:0 for s in unique_classes_after}

for i in train_labels_classes:
    samplesPerClass_before[i]+=1
    
    if (class_mapping[i]):
        samplesPerClass_after[class_mapping[i]]+=1

print('Before: ',samplesPerClass_before)
print('After: ',samplesPerClass_after)


plt.figure(figsize=(10,20))

plt.subplot(2, 1, 1)

plt.bar(samplesPerClass_before.keys(), samplesPerClass_before.values(), color='green')
plt.xticks(rotation=45)
plt.title('Before Class Merge')

plt.subplot(2, 1, 2)
plt.bar(samplesPerClass_after.keys(), samplesPerClass_after.values(), color='red')
plt.title('After Class Merge')

# %% [markdown]
# # Step 5 [a-d]

# %% [code]
class BasicLSTM(nn.Module):
    def __init__(self, input_dim, rnn_size, output_dim, num_layers,drop_prob=0.2, bidirectional=False):
        super(BasicLSTM, self).__init__()
        self.bidirectional = bidirectional
        self.feature_size = rnn_size * 2 if self.bidirectional else rnn_size
        self.direction = 2 if self.bidirectional else 1
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.input_dim = input_dim
        self.rnn_size = rnn_size
        self.lstm = nn.LSTM(input_dim,self.rnn_size, num_layers, dropout=drop_prob, batch_first=True, bidirectional=self.bidirectional) # init LSTM
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(self.feature_size, output_dim) #fully Connected 

      
          
    def forward(self, x, lengths,hidden):
        """ 
            x : 3D numpy array of dimension N x L x D
                N: batch index
                L: sequence index
                D: feature index

            lengths: N x 1
         """
              
        
        # We need only the last output (last_timestep method)
        lstm_out, hidden = self.lstm(x, hidden)
        batch_size=x.shape[0]
        lstm_out = lstm_out.contiguous().view(batch_size,-1, self.feature_size)
        out = self.fc(lstm_out)
        #Keep last output per sequence 
        last_outputs = self.last_timestep(out,lengths,self.bidirectional)
        return last_outputs,hidden

    def last_timestep(self, outputs, lengths, bidirectional=False):
        """
            Returns the last output of the LSTM taking into account the zero padding
        """
        if bidirectional:
            forward, backward = self.split_directions(outputs)
            last_forward = self.last_by_index(forward, lengths)
            last_backward = backward[:, 0, :]
            return torch.cat((last_forward, last_backward), dim=-1)
  

        else:
            return self.last_by_index(outputs, lengths)

    @staticmethod
    def split_directions(outputs):
        direction_size = int(outputs.size(-1) / 2)
        forward = outputs[:, :, :direction_size]
        backward = outputs[:, :, direction_size:]
        return forward, backward

    @staticmethod
    def last_by_index(outputs, lengths):
        # Index of the last output for each sequence.
        new_lengths=np.array([length-1 for length in lengths],dtype='long')
        idx = (torch.from_numpy(new_lengths)).view(-1, 1)
        idx = idx.expand(outputs.size(0),outputs.size(2)).unsqueeze(1).to(device)
        return outputs.gather(1, idx).squeeze().to(device)

    def init_hidden(self, batch_size):
        #initialize hidden size 
        weight = next(self.parameters()).data
        hidden = (weight.new(self.direction*self.num_layers, batch_size, self.rnn_size).zero_().to(device),
                      weight.new(self.direction*self.num_layers, batch_size, self.rnn_size).zero_().to(device))
        return hidden

# %% [code]
# Turn on GPU - Manually in this case for Kaggle
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
device = torch.device('cuda')

# %% [code]
# from:  https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/

def train_model(input_dim,output_dim,save_string,data_loader,val_loader=None,category=True,val=True):
    batch_size=32
    rnn_size = 128 #hidden input
    num_layers = 2 

    model = BasicLSTM(input_dim,rnn_size,output_dim,num_layers)
    model.to(device)
    lr=0.0001
    if (category):
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=0.001)
    epochs = 30
    print_every = 200
    valid_loss_min = np.Inf
    loss_values_train = []
    loss_values_val = []
    model.train()
    counter=0



    for i in range(epochs):
        running_loss_train=0.0
        running_loss_val=0.0
        h = model.init_hidden(batch_size)
        train_losses=[]
        for inputs, labels,lengths in data_loader:
            inputs,labels,lengths= inputs.to(device), labels.to(device),lengths.to(device)
            counter += 1
            h = tuple([e.data for e in h])
            model.zero_grad()
            output, h = model(inputs.float(),lengths,h)
            if (category):
                loss = criterion(output.squeeze(), labels.long())
            else:
                loss = criterion(output.squeeze(), labels.float())
            running_loss_train =+ loss.item() * batch_size
            train_losses.append(loss.item())

            loss.backward()
            optimizer.step()
            if (counter%print_every == 0 and val):
                val_h = model.init_hidden(batch_size)
                val_losses = []
                model.eval()
                for inp, lab,lens in val_loader:
                    inp,lab,lens=inp.to(device),lab.to(device),lens.to(device)
                    val_h = tuple([each.data for each in val_h])
                    out, val_h = model(inp.float(),lens, val_h)
                    val_loss = criterion(out.squeeze(), lab.long())
                    val_losses.append(val_loss.item())
                    #running_loss_val=+ val_loss.item() * batch_size
                model.train()
                #'./state_mel_beat_dict.pt'
                if np.mean(val_losses) <= valid_loss_min:
                    torch.save(model.state_dict(), save_string) # Save model params in Dict
                    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                    valid_loss_min = np.mean(val_losses)
        if val:
            model.eval()
            val_h = model.init_hidden(batch_size) 
            validation_losses=[]
            
            for inp, lab,lens in val_loader:
                inp,lab,lens=inp.to(device),lab.to(device),lens.to(device)
                val_h = tuple([each.data for each in val_h])
                out, val_h = model(inp.float(),lens, val_h)
                val_loss = criterion(out.squeeze(), lab.long())
                validation_losses.append(val_loss.item())
                running_loss_val=+ val_loss.item() * batch_size

            model.train()
        print("Epoch: {}/{}...".format(i+1, epochs),
                "Step: {}...".format(counter),
                "Train Loss: {:.6f}...".format(loss.item()))
        loss_values_train.append(np.mean(train_losses))
        if (val):
            loss_values_val.append(np.mean(validation_losses))    
    plt.plot(range(epochs),loss_values_train)
    if (val):
        plt.plot(range(epochs),loss_values_val)
    if(not val):
        torch.save(model.state_dict(), save_string)

# %% [markdown]
# ## Create Datasets

# %% [code]
#5a 

mel_specs = SpectrogramDatasetCategory(
         '../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/',
         train=True,
         class_mapping=class_mapping,
         max_length=-1,
         read_spec_fn=read_mel_spectrogram)
train_loader_mel, val_loader_mel = torch_train_val_split(mel_specs, 32 ,32, val_size=.33)
test_loader_mel = SpectrogramDatasetCategory(
         '../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/',
         train=False,
         class_mapping=class_mapping,
         max_length=-1,
         read_spec_fn=read_mel_spectrogram)
test_loader_mel = DataLoader(test_loader_mel,
                              batch_size=5,
                              drop_last=True)

# %% [code]
train_model(128,10,'./state_mel_dict.pt',train_loader_mel,val_loader_mel)

# %% [code]
#5b
beat_mel_specs = SpectrogramDatasetCategory(
         '../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/',
         train=True,
         class_mapping=class_mapping,
         max_length=-1,
         read_spec_fn=read_mel_spectrogram)
train_loader_beat_mel, val_loader_beat_mel = torch_train_val_split(beat_mel_specs, 32 ,32, val_size=.33)
test_loader_beat_mel = SpectrogramDatasetCategory(
         '../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/',
         train=False,
         class_mapping=class_mapping,
         max_length=-1,
         read_spec_fn=read_mel_spectrogram)
test_loader_beat_mel = DataLoader(test_loader_beat_mel,
                              batch_size=5,
                              drop_last=True)

# %% [code]
train_model(128,10,'./state_mel_beat_dict.pt',train_loader_beat_mel,val_loader_beat_mel)

# %% [code]
#5c
chroma = SpectrogramDatasetCategory(
         '../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/',
         train=True,
         class_mapping=class_mapping,
         max_length=-1,
         read_spec_fn=read_chromagram)
train_loader_chroma, val_loader_chroma = torch_train_val_split(chroma, 32 ,32, val_size=.33)
test_loader_chroma = SpectrogramDatasetCategory(
         '../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/',
         train=False,
         class_mapping=class_mapping,
         max_length=-1,
         read_spec_fn=read_chromagram)
test_loader_chroma = DataLoader(test_loader_chroma,
                              batch_size=5,
                              drop_last=True)

# %% [code]
train_model(12,10,'./chroma_dict.pt',train_loader_chroma,val_loader_chroma)

# %% [code]
#5d
fused = SpectrogramDatasetCategory(
         '../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/',
         train=True,
         class_mapping=class_mapping,
         max_length=-1,
         read_spec_fn=read_fused_spectrogram)
train_loader_fused, val_loader_fused = torch_train_val_split(fused, 32 ,32, val_size=.33)
test_loader_fused = SpectrogramDatasetCategory(
         '../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/',
         train=False,
         class_mapping=class_mapping,
         max_length=-1,
         read_spec_fn=read_fused_spectrogram)
test_loader_fused = DataLoader(test_loader_fused,
                              batch_size=5,
                              drop_last=True)

# %% [code]
train_model(140,'./fused_dict.pt',train_loader_fused,val_loader_fused)

# %% [markdown]
# # Step 6

# %% [code]
def test(input_dim,save_file,test_loader):
    model = BasicLSTM(input_dim,128,10,2)
    model.to(device)
    criterion=nn.CrossEntropyLoss()
    model.load_state_dict(torch.load(save_file)) #Load System's params (dictionary)
    batch_size=5
    test_losses = []
    num_correct = 0
    h = model.init_hidden(batch_size)
    correct=0
    y_pred_test=[]
    y_true=[]
    model.eval()
    for inputs, labels,lengths in test_loader:
          inputs,labels,lengths= inputs.to(device), labels.to(device),lengths.to(device)
          h = tuple([each.data for each in h])
          output, h = model(inputs.float(),lengths, h)
          test_loss = criterion(output.squeeze(), labels.long())
          test_losses.append(test_loss.item())
          pred = output.data.max(1)[1]  # get the index of the max log-probability
          y_pred_test.append(pred.tolist())
          y_true.append(labels.tolist())
          correct += pred.eq(labels.data).sum()

    print('\nTest set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
              np.mean(test_losses), correct, len(test_loader.dataset),
              100. * correct / (len(test_loader.dataset))))

    #test the model using all the different metrics (precision,recall, F1 score for all the classes)
    print(np.array(y_true).shape)
    print(np.array(y_pred_test).shape)
    from sklearn.metrics import classification_report
    print(classification_report(np.array(y_true).flatten(),np.array(y_pred_test).flatten()))

# %% [code]
#spectograms as input
test(128,'./state_mel_dict.pt',test_loader_mel)

# %% [code]
#beat-synced spectograms as input
test(128,'./state_mel_beat_dict.pt',test_loader_beat_mel)

# %% [code]
#chromagrams as input
test(12,'./chroma_dict.pt',test_loader_chroma)

# %% [code]
#concatenated spectograms and chromograms as input
test(140,'./fused_dict.pt',test_loader_fused)

# %% [markdown]
# # Step 7

# %% [code]
#b
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

#CNN initinialization
class myCNN(nn.Module):
    def __init__(self):
        super(myCNN,self).__init__()
        self._cnn_module = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4),
            Flatten()
        
        )
        self._fc_module = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=10240, out_features=1)
        )
        #Weights Initialization
        #Weights only from the linear and convolutional layer
        
    def forward(self, x):
        x=x.transpose(1,2) # dims [batch_size,1,]
        x=torch.unsqueeze(x,1)
        for layer in self._cnn_module:
            x = layer(x)
        
        for layer in self._fc_module:
            x=layer(x)
        return x
    def initialize_weights(self,layer)->None:
        if isinstance(layer, nn.Conv2d):
            #weights based on Gaussian (mean=0, std = sqrt(2/#Neurons))
            nn.init.kaiming_uniform_(layer.weight)
        elif isinstance(layer, nn.Linear):
            #we use xavier initialization
            nn.init.xavier_uniform_(layer.weight)

# %% [code]
#c
def train_cnn(data_loader,save_string,epochs,category=False):
    model_cnn = myCNN()
    if category:
        model_cnn._fc_module = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=10240, out_features=10)
        )
    
    batch_size=16
    output_dim = 1

    model_cnn.to(device)
    lr=0.001
    if category:
        criterion=nn.CrossEntropyLoss()
    else:
        criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model_cnn.parameters(), lr=lr,weight_decay=0.0001)

    train_loss_min = np.Inf
    loss_values_train = []
    loss_values_val = []
    model_cnn.train()
    counter=0
    


    model_cnn.train()    
    for i in range(epochs):
        running_loss_train=0.0
        running_loss_val=0.0
        train_losses=[]
        for inputs, labels,lengths in data_loader:
            inputs,labels,lengths= inputs.to(device), labels.to(device),lengths.to(device)
            counter += 1
            model_cnn.zero_grad()
            output =  model_cnn(inputs.float())
            if category:
                loss = criterion(output.squeeze(), labels.long())
            else:
                loss = criterion(output.squeeze(), labels.float())
            running_loss_train =+ loss.item() * batch_size
            train_losses.append(loss.item())

            loss.backward()

            optimizer.step()


        model_cnn.eval()
        model_cnn.train()
        print("Epoch: {}/{}...".format(i+1, epochs),
                "Step: {}...".format(counter),
                "Train Loss: {:.6f}...".format(loss.item()))
        loss_values_train.append(np.mean(train_losses))
        if np.mean(train_losses)<train_loss_min:
            torch.save(model_cnn.state_dict(),save_string)
    plt.plot(range(epochs),loss_values_train)
    #torch.save(model_cnn.state_dict(),save_string)

# %% [code]
train_cnn(train_loader_mel,'state_mel_cnn.pt',epochs=45,category=True)

# %% [code]
#d
from sklearn.metrics import classification_report

def cnn_test(save_file,test_loader):
    criterion=nn.CrossEntropyLoss()
    model_cnn = myCNN()
    model_cnn._fc_module = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=10240, out_features=10)
    )
    model_cnn.to(device)
    model_cnn.load_state_dict(torch.load(save_file))
    batch_size=5
    test_losses = []
    num_correct = 0
    correct=0
    y_pred_test=[]
    y_true=[]
    model_cnn.eval()
    for inputs, labels,lengths in test_loader:
          inputs,labels,lengths= inputs.to(device), labels.to(device),lengths.to(device)
          output = model_cnn(inputs.float())
          test_loss = criterion(output.squeeze(), labels.long())
          test_losses.append(test_loss.item())
          pred = output.data.max(1)[1]  # get the index of the max log-probability
          y_pred_test.append(pred.tolist())
          y_true.append(labels.tolist())
          correct += pred.eq(labels.data).sum()

    print('\nTest set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
              np.mean(test_losses), correct, len(test_loader.dataset),
              100. * correct / (len(test_loader.dataset))))
    print(classification_report(np.array(y_true).flatten(),np.array(y_pred_test).flatten()))
cnn_test('./state_mel_cnn.pt',test_loader_mel)

# %% [markdown]
# # Step 8

# %% [code]
#8a
#create a pytorch dataset for the mel spectogram
mel_specs_energy = SpectrogramDataset(
         '../input/patreco3-multitask-affective-music/data/multitask_dataset_beat/',
         train=True,
         task='energy',
         max_length=-1,
         read_spec_fn=read_mel_spectrogram)
train_loader_energy, test_loader_energy= torch_train_val_split(mel_specs_energy, 32 ,32, val_size= 0.2 )

mel_specs_valence = SpectrogramDataset(
         '../input/patreco3-multitask-affective-music/data/multitask_dataset_beat/',
         train=True,
         task='valence',
         max_length=-1,
         read_spec_fn=read_mel_spectrogram)
train_loader_valence, test_loader_valence = torch_train_val_split(mel_specs_valence, 32 ,32, val_size= 0.2 )

mel_specs_danceability = SpectrogramDataset(
         '../input/patreco3-multitask-affective-music/data/multitask_dataset_beat/',
         train=True,
         task='danceability',
         max_length=-1,
         read_spec_fn=read_mel_spectrogram)
train_loader_danceability, test_loader_danceability = torch_train_val_split(mel_specs_danceability, 32 ,32, val_size= 0.2 )

# %% [code]
#8b
train_cnn(train_loader_energy,'state_mel_energy_cnn.pt',epochs=30)

# %% [code]
#8c
train_cnn(train_loader_valence,'state_mel_valence_cnn.pt',epochs=36)

# %% [code]
#8d
train_cnn(train_loader_danceability,'state_mel_danceability_cnn.pt',epochs=30)

# %% [code]
#8e
import scipy.stats


def lstm_model_multitask(test_loader,save_file):
    model_lstm = BasicLSTM (128,128,1,2)
    model_lstm.to(device)
    model_lstm.load_state_dict(torch.load(save_file))
    batch_size=32
    h = model_lstm.init_hidden(batch_size)
    test_losses = []
    y_pred_test=[]
    y_true=[]
    model_lstm.eval()
    for inputs, labels,lengths in test_loader:
          inputs,labels,lengths= inputs.to(device), labels.to(device),lengths.to(device)
          h = tuple([each.data for each in h])
          output, h = model_lstm(inputs.float(),lengths, h)
          y_pred_test.append(output.data.tolist())
          y_true.append(labels.tolist())
    rho= scipy.stats.spearmanr(np.array(y_true).flatten(),np.array(y_pred_test).flatten()).correlation
    print('\nTest set: Spearman Correlation: {:.6f} \n'.format(rho))
    return rho

r_energy=lstm_model_multitask(test_loader_energy,'./state_mel_energy_multi.pt')
r_valence=lstm_model_multitask(test_loader_valence,'./state_mel_valence_multi.pt')
r_danceability = lstm_model_multitask(test_loader_danceability,'./state_mel_danceability_multi.pt')
print((r_energy+r_valence+r_danceability)/3)